## Imports

In [199]:
import numpy as np

## Functions

In [200]:
def Table_Heat(Re):
    if Re <= 10:
        Ch = 0.718
        y = 0.349
    elif Re > 10:
        Ch = 0.348
        y = 0.663
    return [Ch,y]

def Table_Press(Re):
    if Re < 10:
        Kp = 50
        z = 1
    elif Re >= 10 and Re <= 100:
        Kp = 19.40
        z = 0.589
    elif Re > 100:
        Kp = 2.990
        z = 0.183
    return [Kp,z]

def Visco_water(T):
    Visc = (3e-08*T**4 - 9e-06*T**3 + 0.001*T**2 - 0.0552*T + 1.7784) # cPoise
    return Visc

def Visco_egg(T):
    Visc = 21.325*np.exp(-0.022*T) # cPoise
    return Visc

def density_egg(T):
    rho = -0.0036*T**2-0.272*T+1039.9 # kg/m^3
    return rho

def density_water(T):
    rho = -0.0036*T**2 - 0.0656*T + 1000.4 # kg/m^3
    return rho

## Parameters

In [238]:
T_r1 = 8
T_r2 = 35

T_r3 = 60
T_r4 = 33

T_h1 = T_r2
T_h2 = 63

T_h3 = 80
T_h4 = 60

T_c1 = 1
T_c2 = 20

T_c3 = T_r4
T_c4 = 5

Mass_Egg = 35000 # kg/h
Mass_Hwater = 30000 # kg/h
Mass_Cwater = 34000 # kg/h

LMTD_R = T_r2 - T_r1
LMTD_H = ((T_h4 - T_h1) - (T_h3 - T_h2))/np.log((T_h4 - T_h1)/(T_h3 - T_h2))
LMTD_C = ((T_c4 - T_c1) - (T_c3 - T_c2))/np.log((T_c4 - T_c1)/(T_c3 - T_c2))

print (LMTD_R,LMTD_H,LMTD_C)

27 20.7435267832 7.63582221085


## Properties

In [202]:
# Egg 

Cp_egg = 2.68 # KJ/Kg K
k_egg = 0.40

# Water for Heating

mu_Hwater = Visco_water((T_h3 + T_h4)/2)
Cp_Hwater = 4.187
k_Hwater = 0.4
rho_Hwater = density_water((T_h3 + T_h4)/2)

mu_bHwater = Visco_water((T_h3 + T_h4)/2)
mu_wHwater = Visco_water((T_h3 + T_h4)/2 - 2)

# Water for Cooling

mu_Cwater = Visco_water((T_c1 + T_c2)/2)
Cp_Cwater = 4.187
k_Cwater = 0.397
rho_Cwater = density_water((T_c1 + T_c2)/2)

mu_bCwater = Visco_water((T_c1 + T_c2)/2)
mu_wCwater = Visco_water((T_c1 + T_c2)/2 + 2)

# Stainless Steel AISI 316

Kw = 16.3

# Fouling Factors

R_fh = 0.0002
R_fc = 0.0005

## Regenerative PHE

### Plate design

In [217]:
Dp = 0.08 # m
Lv = 1.095 # m
Lh = 0.22 # m
t = 0.6/1000 # mm
Beta = 30 
A1 = 0.38 # m
b = 0.004 # m

Lp = Lv - Dp
Lw = Lh + Dp

A_1p = Lp*Lw

phi = A1/A_1p
De = 2*b/phi
print (De)

Np = 2 # Inputs
Nt = 112

0.006410526315789472


## Nusselt and heat capacity 

### Hot side

In [218]:
mu_egg = Visco_egg((T_r3 + T_r4)/2)
mu_wegg = Visco_egg((T_r3 + T_r4)/2 - 2)
rho_egg = density_egg((T_r3 + T_r4)/2)

Ncp = (Nt - 1)/(2*Np)
Gc = (Mass_Egg/3600)/(Ncp*b*Lw)
Re = Gc*De/(mu_egg/1000)

Pr = Cp_egg*(mu_egg)/k_egg

C_h,y = Table_Heat(Re)
J_h = C_h*Re**y

Nu = J_h*(Pr**0.33)*(mu_egg/mu_wegg)**0.17
H_h = Nu*k_egg/De

print (Nu,H_h)

48.5109995347 3026.95892006


## Pressure drop

### Hot side

In [219]:
Kp,z = Table_Press(Re)
f = Kp/(Re**z)

Pc = (4*f*Lv*Np*Gc**2)/(De*2*rho_egg)*(mu_egg/mu_wegg)**-0.17

Gp = (Mass_Egg/3600)/((np.pi/4)*Dp**2)

Pp = 1.4*(Gp**2)/(2*rho_egg)*Np
Tot_pressure = Pc + Pp
print (Tot_pressure/1000)

68.0643836669


### Cold side

In [220]:
mu_egg = Visco_egg((T_r1 + T_r2)/2)
mu_wegg = Visco_egg((T_r1 + T_r2)/2 + 2)
rho_egg = density_egg((T_r1 + T_r2)/2)

Ncp = (Nt - 1)/(2*Np)
Gc = (Mass_Egg/3600)/(Ncp*b*Lw)
Re = Gc*De/(mu_egg/1000) 

Pr = Cp_egg*mu_egg/k_egg

C_h,y = Table_Heat(Re)
J_h = C_h*Re**y

Nu = J_h*(Pr**0.33)*(mu_egg/mu_wegg)**0.17
H_c = Nu*k_egg/De

print (Nu,H_c)

41.0011685189 2558.36519494


### Cold side

In [221]:
Kp,z = Table_Press(Re)

f = Kp/Re**z

Pc = (4*f*Lv*Np*Gc**2)/(De*2*rho_egg)*(mu_egg/mu_wegg)**-0.17

Gp = (Mass_Egg/3600)/((np.pi/4)*Dp**2)

Pp = 1.4*(Gp**2)/(2*rho_egg)*Np

Tot_pressure = Pc + Pp
print (Tot_pressure/1000)

72.7721053332


## Heat Transfer Calculations

In [222]:
U_f = 1/(1/H_h + 1/H_c + t/Kw + (R_fh + R_fc)*1.2)
Qr = (Mass_Egg/3600)*Cp_egg*(T_r2 - T_r1)

Ne = Nt-2
Ae = A1*Ne
F = 1

Qf = U_f*Ae*F*LMTD_R

C = Qf/(Qr*1000)
print (C)

1.00388936107


## Flowrate Correction

In [223]:
Mass_egg = (Qf/1000)/(Cp_egg*((T_r2 - T_r1)))*3600
Mass_egg

35136.127637492275

## Heating PHE

## Nusselt and heat capacity 

### Hot side

In [224]:
Np = 2 # Inputs
Nt = 128

In [225]:
Ncp = (Nt - 1)/(2*Np)
Gc = (Mass_Hwater/3600)/(Ncp*b*Lw)
Re = Gc*De/(mu_Hwater/1000)

Pr = Cp_Hwater*(mu_Hwater)/k_Hwater

C_h,y = Table_Heat(Re)
J_h = C_h*Re**y

Nu = J_h*(Pr**0.33)*(mu_bHwater/mu_wHwater)**0.17
H_h = Nu*k_Hwater/De

print (Nu,H_h)

119.84256787508771 7477.8613780842925


## Pressure drop

### Hot side

In [226]:
Kp,z = Table_Press(Re)
f = Kp/(Re**z)

Pc = (4*f*Lv*Np*Gc**2)/(De*2*rho_Hwater)*(mu_bHwater/mu_wHwater)**-0.17

Gp = (Mass_Hwater/3600)/((np.pi/4)*Dp**2)

Pp = 1.3*(Gp**2)/(2*rho_Hwater)*Np
Tot_pressure = Pc + Pp
print (Tot_pressure/1000)

26.664687407984378


### Cold side

In [227]:
mu_egg = Visco_egg((T_h1 + T_h2)/2)
mu_wegg = Visco_egg((T_h1 + T_h2)/2 + 2)
rho_egg = density_egg((T_h1 + T_h2)/2)

Ncp = (Nt - 1)/(2*Np)
Gc = (Mass_Egg/3600)/(Ncp*b*Lw)
Re = Gc*De/(mu_egg/1000) 

Pr = Cp_egg*mu_egg/k_egg

C_h,y = Table_Heat(Re)
J_h = C_h*Re**y

Nu = J_h*(Pr**0.33)*(mu_egg/mu_wegg)**0.17
H_c = Nu*k_egg/De

print (Nu,H_c)

45.8689332275 2862.10092388


### Cold side

In [228]:
Kp,z = Table_Press(Re)

f = Kp/Re**z

Pc = (4*f*Lv*Np*Gc**2)/(De*2*rho_egg)*(mu_egg/mu_wegg)**-0.17

Gp = (Mass_Egg/3600)/((np.pi/4)*Dp**2)

Pp = 1.3*(Gp**2)/(2*rho_egg)*Np

Tot_pressure = Pc + Pp
print (Tot_pressure/1000)

52.9020827684


## Heat Transfer Calculations

In [229]:
Kw = 16.3
R_fh = 0.0002
R_fc = 0.0005

U_f = 1/(1/H_h + 1/H_c + t/Kw +(R_fh + R_fc)*1.2)
Qr = (Mass_Egg/3600)*Cp_egg*(T_h2 - T_h1)

Ne = Nt-2
Ae = A1*Ne
F = 1

Qf = U_f*Ae*F*LMTD_H

C = Qf/(Qr*1000)
print (C)

1.00106277116


## Flowrate Correction

In [230]:
Mass_egg = ((Qf/1000)/(Cp_egg*((T_h2 - T_h1)))*3600)
Mass_water = ((Qf/1000)/(Cp_Hwater*((T_h3 - T_h4)))*3600)
print (Mass_egg,Mass_water)

35037.1969908 31397.0774085


## Cooling PHE

## Nusselt and heat capacity

### Hot side

In [231]:
Np = 4 # Inputs
Nt =448

In [232]:
mu_egg = Visco_egg((T_c3 + T_c4)/2)
mu_wegg = Visco_egg((T_c3 + T_c4)/2 - 2)
rho_egg = density_egg((T_c3 + T_c4)/2)

Ncp = (Nt - 1)/(2*Np)
Gc = (Mass_Egg/3600)/(Ncp*b*Lw)
Re = Gc*De/(mu_egg/1000) 

Pr = Cp_egg*mu_egg/k_egg

C_h,y = Table_Heat(Re)
J_h = C_h*Re**y

Nu = J_h*(Pr**0.33)*(mu_egg/mu_wegg)**0.17
H_h = Nu*k_egg/De

print (Nu,H_h)

24.9357950331 1555.9280973


## Pressure drop

### Hot side

In [233]:
Kp,z = Table_Press(Re)
f = Kp/Re**z

Pc = (4*f*Lv*Np*Gc**2)/(De*2*rho_egg)*(mu_egg/mu_wegg)**-0.17

Gp = (Mass_Egg/3600)/((np.pi/4)*Dp**2)

Pp = 1.3*(Gp**2)/(2*rho_egg)*Np

Tot_pressure = Pc + Pp
print (Tot_pressure/1000)

55.3951100207


### Cold side

In [234]:
Ncp = (Nt - 1)/(2*Np)
Gc = (Mass_Cwater/3600)/(Ncp*b*Lw)
Re = Gc*De/(mu_Cwater/1000)

Pr = Cp_Cwater*(mu_Cwater)/k_Cwater

C_h,y = Table_Heat(Re)
J_h = C_h*Re**y

Nu = J_h*(Pr**0.33)*(mu_bCwater/mu_wCwater)**0.17
H_c = Nu*k_Cwater/De

print (Nu,H_c)

63.84786940551802 3954.0597612957517


### Cold side

In [235]:
Kp,z = Table_Press(Re)
f = Kp/(Re**z)

Pc = (4*f*Lv*Np*Gc**2)/(De*2*rho_Cwater)*(mu_bCwater/mu_wCwater)**-0.17

Gp = (Mass_Cwater/3600)/((np.pi/4)*Dp**2)

Pp = 1.3*(Gp**2)/(2*rho_Cwater)*Np
Tot_pressure = Pc + Pp
print (Tot_pressure/1000)

33.44427121606136


## Heat Transfer Calculations

In [236]:
Kw = 16.3
R_fh = 0.0002
R_fc = 0.0005

U_f = 1/(1/H_h + 1/H_c + t/Kw + (R_fh + R_fc)*1.2)
Qr = (Mass_Egg/3600)*Cp_egg*(T_c3 - T_c4)

Ne = Nt-2
Ae = A1*Ne
F = 1

Qf = U_f*Ae*F*LMTD_C
C = Qf/(Qr*1000)
print (C)

1.00080582691


## Flowrate Correction

In [237]:
Mass_egg = ((Qf/1000)/(Cp_egg*((T_c3 - T_c4)))*3600)
Mass_water = ((Qf/1000)/(Cp_Cwater*((T_c2 - T_c1)))*3600)
print (Mass_egg,Mass_water)

35028.2039418 33041.0722888
